### AirBnb Rio

Installing libs

In [162]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
%matplotlib inline

Reading data

In [163]:
df = pd.read_csv('./listings.csv')
df.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20210321015201,2021-03-25,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,https://www.airbnb.com/users/show/68997,...,10.0,10.0,9.0,NaN,t,1,1,0,0,2.00
1,24480,https://www.airbnb.com/rooms/24480,20210321015201,2021-03-22,Nice and cozy near Ipanema Beach,My studio is located in the best of Ipanema. ...,"The beach, the lagoon, Ipanema is a great loca...",https://a0.muscache.com/pictures/11955612/b28e...,99249,https://www.airbnb.com/users/show/99249,...,10.0,10.0,9.0,NaN,f,1,1,0,0,0.67
2,35636,https://www.airbnb.com/rooms/35636,20210321015201,2021-03-25,Cosy flat close to Ipanema beach,This cosy apartment is just a few steps away ...,The apartment street is very quiet and safe ....,https://a0.muscache.com/pictures/20009355/38b6...,153232,https://www.airbnb.com/users/show/153232,...,10.0,10.0,9.0,NaN,f,1,1,0,0,2.00
3,35764,https://www.airbnb.com/rooms/35764,20210321015201,2021-03-25,COPACABANA SEA BREEZE - RIO - 20 X Superhost,Our newly renovated studio is located in the b...,Our guests will experience living with a local...,https://a0.muscache.com/pictures/23782972/1d3e...,153691,https://www.airbnb.com/users/show/153691,...,10.0,10.0,10.0,NaN,f,1,1,0,0,2.79
4,41198,https://www.airbnb.com/rooms/41198,20210321015201,2021-03-22,"Modern 2bed,Top end of Copacabana","<b>The space</b><br />Stay in this, Modern,cle...",NaN,https://a0.muscache.com/pictures/3576716/2d6a9...,178975,https://www.airbnb.com/users/show/178975,...,9.0,9.0,9.0,NaN,f,2,2,0,0,0.18


#### Neighbourhood Overview

1. Top neighbourhoods in number of properties

In [164]:
fig = px.bar(df.neighbourhood_cleansed.value_counts().nlargest(10), 
             orientation='h', 
             title="Top 10 neighbourhoods by number of properties",
            template='simple_white')
fig.update_yaxes(categoryorder='total ascending')
fig.update_layout(showlegend=False)

2. Top neighbourhoods by highest prices

In [165]:
df['price'] = df['price'].str.extract(r'(\d+)')
df['price'] = df['price'].astype('float64').astype('Int64')

In [175]:
fig = px.histogram(df['price'])
fig.show()

In [170]:
fig = px.bar(df.groupby('neighbourhood_cleansed').median()['price'].nlargest(15), 
             orientation='h', 
             title="Top 10 neighbourhoods by highest prices",
            template='simple_white')
fig.update_yaxes(categoryorder='total ascending')
fig.update_layout(showlegend=False)